In [7]:
import arcpy
from arcpy.sa import ZonalStatisticsAsTable
import uuid

# ==== 直接指定輸入參數 ====
polygon_fc = r"D:\ArcGIS\Morpho_Spatial_Demography\MSD_Taiwan.gdb\OL_P3_2204_corals_taglab"   # Polygon FeatureClass
raster = "OL_P3_2204_SAML_forms"       # Raster 檔案
zone_field = "OBJECTID"                     # 分區欄位
out_field = "Zonal_form"                   # 結果欄位名稱

arcpy.CheckOutExtension("Spatial")

# 環境對齊，避免取樣偏移
arcpy.env.snapRaster = raster
arcpy.env.cellSize = raster

# —— 決定欄位型別（若取不到 pixelType 就預設整數） ——
try:
    pixel_type = arcpy.Raster(raster).pixelType.upper()
except Exception:
    pixel_type = "U8"  # fallback
float_types = {"F32", "F64"}
field_type = "DOUBLE" if any(t in pixel_type for t in float_types) else "LONG"

# 若目標欄位不存在就新增
if out_field not in [f.name for f in arcpy.ListFields(polygon_fc)]:
    arcpy.management.AddField(polygon_fc, out_field, field_type)

# —— 用 scratch GDB 當暫存輸出，較穩定 ——
scratch_gdb = arcpy.env.scratchGDB  # e.g., C:\Users\...\AppData\Local\Temp\scratch.gdb
tmp_name = f"zst_{uuid.uuid4().hex[:8]}"
tmp_tbl = os.path.join(scratch_gdb, tmp_name)

try:
    # 先刪掉同名殘留（極少數狀況）
    if arcpy.Exists(tmp_tbl):
        arcpy.management.Delete(tmp_tbl)

    # 執行 ZonalStatisticsAsTable
    ZonalStatisticsAsTable(
        in_zone_data=polygon_fc,
        zone_field=zone_field,
        in_value_raster=raster,
        out_table=tmp_tbl,
        ignore_nodata="DATA",
        statistics_type="MAJORITY"
    )

    # 確認輸出表真的存在；若沒有，就把工具訊息印出來協助定位
    if not arcpy.Exists(tmp_tbl):
        print("⚠️ ZonalStatisticsAsTable 沒有產生輸出表。工具訊息如下：")
        print(arcpy.GetMessages())
        raise RuntimeError("ZonalStatisticsAsTable failed: output table not created.")

    # 有些情況（特別是文字型 zone_field）輸出欄位名稱可能被截斷/轉名
    # 安全作法：先列出欄位，再找「最像」的區域欄位；但大多數情況直接用原名即可。
    out_fields = [f.name for f in arcpy.ListFields(tmp_tbl)]
    if zone_field not in out_fields:
        # 嘗試以不分大小寫比對
        candidates = {f.lower(): f for f in out_fields}
        z_field_in_tbl = candidates.get(zone_field.lower(), None)
        if not z_field_in_tbl:
            # 再退而求其次找前綴相同者（避免被截斷時對不上）
            z_field_in_tbl = next((f for f in out_fields if f.upper().startswith(zone_field.upper()[:8])), None)
            if not z_field_in_tbl:
                raise RuntimeError(f"找不到輸出表的分區欄位。原欄位名：{zone_field}；輸出表欄位：{out_fields}")
    else:
        z_field_in_tbl = zone_field

    # 檢查 MAJORITY 欄位是否存在
    if "MAJORITY" not in out_fields:
        raise RuntimeError(f"輸出表沒有 MAJORITY 欄位。實際欄位：{out_fields}")

    # 讀成 {zone: majority} 對照
    z2maj = {}
    with arcpy.da.SearchCursor(tmp_tbl, [z_field_in_tbl, "MAJORITY"]) as cur:
        for z, maj in cur:
            z2maj[z] = maj

    # 更新 polygon 的欄位
    with arcpy.da.UpdateCursor(polygon_fc, [zone_field, out_field]) as ucur:
        for row in ucur:
            row[1] = z2maj.get(row[0], None)
            ucur.updateRow(row)

    print(f"完成！已將 {raster} 的多數值寫入 {polygon_fc} 的欄位 {out_field}")

finally:
    # 清理暫存與授權
    try:
        if arcpy.Exists(tmp_tbl):
            arcpy.management.Delete(tmp_tbl)
    except Exception:
        pass
    arcpy.CheckInExtension("Spatial")

完成！已將 OL_P3_2204_SAML_forms 的多數值寫入 D:\ArcGIS\Morpho_Spatial_Demography\MSD_Taiwan.gdb\OL_P3_2204_corals_taglab 的欄位 Zonal_form


In [10]:
import arcpy
import os

# === 輸入 ===
gdb_path = r"D:\ArcGIS\Morpho_Spatial_Demography\MSD_Taiwan.gdb"
fc_name = "OL_P3_2204_corals_taglab"     # FeatureClass 名稱（不含路徑）
code_field = "Zonal_form"                # 代碼欄位 (LONG)
text_field = "Zonal_form_txt"            # 轉文字的欄位
fallback_domain_name = "Growth_form"     # 若欄位未掛 domain，就用這個名字

# === 指定工作空間 & FC 路徑 ===
arcpy.env.workspace = gdb_path
fc_path = os.path.join(gdb_path, fc_name)

if not arcpy.Exists(fc_path):
    raise RuntimeError(f"找不到 FeatureClass：{fc_path}")

# === 找出欄位實際的 domain 名稱（若沒有就用 fallback）===
desc = arcpy.Describe(fc_path)
actual_domain_name = None
for f in desc.fields:
    if f.name == code_field:
        actual_domain_name = f.domain
        break

domain_name = actual_domain_name or fallback_domain_name
if not domain_name:
    raise RuntimeError(f"{code_field} 欄位沒有 domain，也未提供 fallback 名稱。")

# === 取得該 GDB 內的所有 domain，找到目標 domain ===
domains = arcpy.da.ListDomains(gdb_path)
target_domain = next((d for d in domains if d.name == domain_name), None)
if target_domain is None:
    raise RuntimeError(f"在 {gdb_path} 找不到名為 {domain_name} 的 domain。")

# === 把 domain 做成對照表，兼容字串鍵與整數鍵 ===
# 例：codedValues 可能像 {'1': '住宅區', '2': '商業區'}（文字 code）
#    或 {1: '住宅區', 2: '商業區'}（數字 code）
domain_dict = {}
for k, v in target_domain.codedValues.items():
    # 原始鍵
    domain_dict[k] = v
    # 加入字串鍵
    domain_dict[str(k)] = v
    # 若可轉成整數，也加入整數鍵（避免 '1' 與 1 對不起來）
    try:
        domain_dict[int(k)] = v
    except (TypeError, ValueError):
        pass

# === 若文字欄位不存在就新增 ===
if text_field not in [f.name for f in arcpy.ListFields(fc_path)]:
    arcpy.management.AddField(fc_path, text_field, "TEXT", field_length=255)

# === 批次更新 ===
n_total = 0
n_hit = 0
with arcpy.da.UpdateCursor(fc_path, [code_field, text_field]) as cur:
    for row in cur:
        code_val = row[0]
        n_total += 1

        # 多策略比對：原值、字串化、（若可）整數化
        mapped = domain_dict.get(code_val)
        if mapped is None and code_val is not None:
            mapped = domain_dict.get(str(code_val))
        if mapped is None and isinstance(code_val, str):
            try:
                mapped = domain_dict.get(int(code_val))
            except ValueError:
                pass

        row[1] = mapped
        if mapped is not None:
            n_hit += 1
        cur.updateRow(row)

print(f"完成！已將 {code_field} 依 domain「{domain_name}」轉到 {text_field}。")
print(f"總筆數：{n_total}，成功對應：{n_hit}，未命中：{n_total - n_hit}")


完成！已將 Zonal_form 依 domain「Growth_form」轉到 Zonal_form_txt。
總筆數：423，成功對應：252，未命中：171


In [18]:
# 計算每個 50×50cm 矩形內 hot 的百分比（SUM / Shape_Area）
import arcpy
from arcpy.sa import Raster, Con, IsNull, TabulateArea   # ← 這裡很關鍵！
import os

arcpy.CheckOutExtension("Spatial")

# === 參數（改成你的實際路徑/欄位）===
grid_fc      = r"D:\ArcGIS\Morpho_Spatial_Demography\MSD_Hawaii.gdb\k2b_grid_50cm"   # 50x50cm Polygon FC
value_raster  = r"D:\ArcGIS\Morpho_Spatial_Demography\MSD_Tajao.gdb\k2b_2025_hot_filtered"  # 1/NoData 或 1/0
out_gdb      = r"D:\ArcGIS\Morpho_Spatial_Demography\MSD_Hawaii.gdb"                      # 輸出表放這裡
out_tbl       = os.path.join(out_gdb, "hot_sum_by_grid")         # Zonal 統計表
zone_field    = arcpy.Describe(grid_fc).OIDFieldName             # 用 OID 作 zone ID
shape_area_f  = "Shape_Area"                                     # 直接用現成的面積欄位
pct_field     = "hot_pct"                                        # 百分比欄位名稱（新建）

# === 檢查資料 ===
for p in [grid_fc, hot_ras_path, out_gdb]:
    if not arcpy.Exists(p):
        raise ValueError(f"找不到資料：{p}")

hot = Raster(hot_ras_path)
arcpy.env.snapRaster = hot
arcpy.env.cellSize   = hot
arcpy.env.extent     = hot

# === 二值化 hot raster (1=有值, 0=無值)
hot_bin = Con(IsNull(hot), 0, 1)

# === Tabulate Area ===
zone_field = arcpy.Describe(grid_fc).OIDFieldName
if arcpy.Exists(out_tbl):
    arcpy.management.Delete(out_tbl)
TabulateArea(grid_fc, zone_field, hot_bin, "Value", out_tbl, hot.meanCellWidth)

# === 檢查欄位名稱 ===
tab_fields = [f.name for f in arcpy.ListFields(out_tbl)]
print("TabulateArea 欄位:", tab_fields)

# 嘗試自動找到對應 hot 面積欄位
hot_area_field = None
for candidate in ["AREA_1", "VALUE_1"]:
    if candidate in tab_fields:
        hot_area_field = candidate
        break

if not hot_area_field:
    raise ValueError("⚠️ 找不到 AREA_1 或 VALUE_1 欄位，請確認 raster 的 hot 區域值為 1。")

print(f"✅ 使用欄位：{hot_area_field}")

# === Join 面積結果回格網 ===
arcpy.management.JoinField(grid_fc, zone_field, out_tbl, zone_field, [hot_area_field])

# === 建立/計算百分比欄位 ===
if pct_field not in [f.name for f in arcpy.ListFields(grid_fc)]:
    arcpy.management.AddField(grid_fc, pct_field, "DOUBLE")

arcpy.management.CalculateField(
    grid_fc, pct_field,
    expression=f"pct(!{hot_area_field}!, !{shape_area_f}!)",
    expression_type="PYTHON3",
    code_block="""
def pct(area_hot, area_total):
    if area_hot in (None,):
        area_hot = 0
    if area_total in (None, 0):
        return None
    return (area_hot / area_total) * 100.0
"""
)

print(f"✅ 完成！百分比欄位 hot_pct 已寫入 {grid_fc}。")

TabulateArea 欄位: ['OBJECTID', 'OBJECTID_1', 'VALUE_0', 'VALUE_1']
✅ 使用欄位：VALUE_1
✅ 完成！百分比欄位 hot_pct 已寫入 D:\ArcGIS\Morpho_Spatial_Demography\MSD_Hawaii.gdb\k2b_grid_50cm。
